In [8]:
import pandas as pd


In [9]:
input_file_path = '/Users/thebekhruz/Desktop/100Days-Of-Code/100-Days-of-NLP-Odyssey/data/raw/sample_2k_swop_output_updated.jsonl'
output_file_path_titles = 'data/processed_data/processed_data_title.csv'
output_file_path_descriptions = 'data/processed_data/processed_data_descr.csv'

In [10]:
df = pd.read_csv(input_file_path, delimiter='\t', header=None, names=['doc_id', 'type', 'value'])
df.head(10)

,doc_id,type,value
0,"{""IAID"": ""a7bb9917-95ff-3f55-a640-4c5afcec25f2...",NaN,NaN
1,"{""IAID"": ""c29a7b77-7c46-3b85-88fe-05c8f4b2e384...",NaN,NaN
2,"{""IAID"": ""196c11e6-f7b6-392f-ae41-28653345087c...",NaN,NaN
3,"{""IAID"": ""7a5aace6-2398-3dcf-8843-37ff6ccea875...",NaN,NaN
4,"{""IAID"": ""c66c4715-c03a-3aab-964b-e733f3ff1cf4...",NaN,NaN
5,"{""IAID"": ""d1159b13-8aa9-35c1-a4c2-fd13e24732b2...",NaN,NaN
6,"{""IAID"": ""e39a291d-ed39-3b56-b9aa-2022964a4114...",NaN,NaN
7,"{""IAID"": ""3b84ea4c-e194-3c34-abbf-064a41ad59da...",NaN,NaN
8,"{""IAID"": ""bf418a27-d1d9-324b-a53f-50ab7ae8d81d...",NaN,NaN
9,"{""IAID"": ""5bc83263-dcd0-3764-98cb-a3761480b4c7...",NaN,NaN


In [11]:
df.isna().value_counts()

doc_id  type  value
False   True  True     2000
Name: count, dtype: int64

In [12]:
df.fillna('None', inplace=True)

/var/folders/xh/x6c57wpx5hv7g4hxw7y4c6cc0000gn/T/ipykernel_10582/3754895673.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)


In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

In [14]:
# Takes around 50 seconds to run with current input of around 2k.
df['DescriptionVector'] = df['value'].apply(lambda x: model.encode(x))

In [15]:
df.head()

,doc_id,type,value,DescriptionVector
0,"{""IAID"": ""a7bb9917-95ff-3f55-a640-4c5afcec25f2...",None,None,"[0.0068389913, 0.017649429, 0.014443066, 0.008..."
1,"{""IAID"": ""c29a7b77-7c46-3b85-88fe-05c8f4b2e384...",None,None,"[0.0068389913, 0.017649429, 0.014443066, 0.008..."
2,"{""IAID"": ""196c11e6-f7b6-392f-ae41-28653345087c...",None,None,"[0.0068389913, 0.017649429, 0.014443066, 0.008..."
3,"{""IAID"": ""7a5aace6-2398-3dcf-8843-37ff6ccea875...",None,None,"[0.0068389913, 0.017649429, 0.014443066, 0.008..."
4,"{""IAID"": ""c66c4715-c03a-3aab-964b-e733f3ff1cf4...",None,None,"[0.0068389913, 0.017649429, 0.014443066, 0.008..."


In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Convert the list of vectors into a matrix for similarity calculation
vector_matrix = np.stack(df['DescriptionVector'].values)




In [17]:
# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(vector_matrix)

# Set the similarity of each document to itself to 0 to avoid self-matching
np.fill_diagonal(similarity_matrix, 0)



In [ ]:
# Takes about 1 min with current input of 2k.

threshold = 0.8  # You can adjust the threshold based on your needs
similar_pairs = []

for i in range(similarity_matrix.shape[0]):
    for j in range(i + 1, similarity_matrix.shape[0]):  # Avoid duplicate pairs (i, j) and (j, i)
        if similarity_matrix[i, j] > threshold:
            similar_pairs.append((df.iloc[i]['doc_id'], df.iloc[j]['doc_id'], similarity_matrix[i, j]))

# Convert similar pairs to a DataFrame for better visualization
similar_pairs_df = pd.DataFrame(similar_pairs, columns=['Doc_ID_1', 'Doc_ID_2', 'Similarity'])



In [19]:
print(similar_pairs_df.head())

                                            Doc_ID_1  \
0  {"IAID": "a7bb9917-95ff-3f55-a640-4c5afcec25f2...   
1  {"IAID": "a7bb9917-95ff-3f55-a640-4c5afcec25f2...   
2  {"IAID": "a7bb9917-95ff-3f55-a640-4c5afcec25f2...   
3  {"IAID": "a7bb9917-95ff-3f55-a640-4c5afcec25f2...   
4  {"IAID": "a7bb9917-95ff-3f55-a640-4c5afcec25f2...   

                                            Doc_ID_2  Similarity  
0  {"IAID": "c29a7b77-7c46-3b85-88fe-05c8f4b2e384...         1.0  
1  {"IAID": "196c11e6-f7b6-392f-ae41-28653345087c...         1.0  
2  {"IAID": "7a5aace6-2398-3dcf-8843-37ff6ccea875...         1.0  
3  {"IAID": "c66c4715-c03a-3aab-964b-e733f3ff1cf4...         1.0  
4  {"IAID": "d1159b13-8aa9-35c1-a4c2-fd13e24732b2...         1.0  


In [33]:
output_file_path = '/Users/thebekhruz/Desktop/100Days-Of-Code/100-Days-of-NLP-Odyssey/simmilarity_df.json'
df.to_json(output_file_path, orient='records')


In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es = Elasticsearch(
    "https://localhost:9200/",
    basic_auth=("elastic", "XFvYbJBlk8zQARYN26ZV"),
    ca_certs="/Users/thebekhruz/Desktop/100Days-Of-Code/elasticsearch-8.12.1/config/certs/http_ca.crt"
)

es.ping()

True

In [ ]:
mapping = {
    "properties": {
        "doc_id": {
            "type": "keyword"   # Suitable for unique identifiers
        },
        "description": {
            "type": "text"      # Suitable for full-text search
        },
        "description_vector": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}


In [ ]:
es.indices.create(index='odyssey', mappings=mapping )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'odyssey'})

In [ ]:
records = df.to_dict(orient='records')

In [ ]:
# records[0]


In [ ]:
for record in records:
    try:
        es.index(index='odyssey', body=record, id=record['doc_id'])
    except Exception as e:
        print(f'An error has occured: {e}')

In [ ]:
input_query = 'Give me all documents which are related to queen victoria'
vectorised_query = model.encode(input_query)

query = {
    "field": "DescriptionVector",
    "query_vector": vectorised_query,
    "k": 5,
    "num_candidates":500
    

}

res = es.knn_search(index='odyssey', knn=query, source=["doc_id", "value"])
res['hits']['hits']

/var/folders/xh/x6c57wpx5hv7g4hxw7y4c6cc0000gn/T/ipykernel_8464/2144712460.py:13: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index='odyssey', knn=query, source=["doc_id", "value"])


[{'_index': 'odyssey',
  '_id': '30ad96fd-bf95-3b9c-9a02-e6c7a323aac7',
  '_score': 0.7996934,
  '_source': {'doc_id': '30ad96fd-bf95-3b9c-9a02-e6c7a323aac7',
   'value': 'appear press report image show queen accompany earl de la warr postmaster general leave queen horace harrison lady margaret hay lord cottesloe hon martin charteris'}},
 {'_index': 'odyssey',
  '_id': 'a32cf70d-a5f8-352f-aa4e-102e09af46cc',
  '_score': 0.7763544,
  '_source': {'doc_id': 'a32cf70d-a5f8-352f-aa4e-102e09af46cc',
   'value': 'upper green street high wycombe queen victoria plaque 1897 show picture victoria surround name country empire'}},
 {'_index': 'odyssey',
  '_id': 'fff8c4c3-f959-31ae-a5a4-2a99e4f3c95c',
  '_score': 0.7742809,
  '_source': {'doc_id': 'fff8c4c3-f959-31ae-a5a4-2a99e4f3c95c',
   'value': 'view plaque commemorate 60 year reign queen victoria queen great britain ireland empress india etc date 1897 set wall house upper green street'}},
 {'_index': 'odyssey',
  '_id': '184f1135-9585-3da0-95f

In [ ]:
df.head()

,doc_id,type,value,DescriptionVector
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,description,east corner queen victoria road junction easto...,"[0.0010302312, -0.023901274, 0.01724779, 0.049..."
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,description,page bucks free press time capsule 7th july 18...,"[-0.01174476, 0.020760344, 0.0038109133, 0.004..."
2,196c11e6-f7b6-392f-ae41-28653345087c,description,new police station high wycombe view opposite ...,"[-0.002487674, 0.0084320465, 0.0564526, 0.0341..."
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,description,corridor entrance reference library,"[0.00631654, 0.0067037106, -0.006011956, 0.020..."
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,description,terrace brick flint cottage,"[-0.057990633, -0.028714327, 0.003651726, 0.02..."
